Importing library psycopg2 for creating a database connection with our PostgreSQL server and creating a cursor for executing SQL queries on the associated database.

In [1]:
import psycopg2 as psy

The below block of code is written to estabilish a connection with our database and create a cursor for the respective connection.

In [2]:
conn=psy.connect("host=127.0.0.1 dbname=db_netflix_snowflake user=postgres password=Netflix@200")
conn.set_session(autocommit=True)
cur=conn.cursor()

Once we have estabilished the connection and created a cursor, we are going to write multiple SQL queries for creating tables, inserting data into these tables and creating a '.csv' file from a query result. These queries would be stored in variables and then we'd use our cursor to execute these queries.

In [3]:
Create_Credits_Table='''
Create Table Credits(
	Person_ID varchar,
	Movie_ID varchar,
	Name_of_the_Person varchar,
	Character_played varchar,
	Role_of_the_person varchar
)
'''

In [4]:
cur.execute(Create_Credits_Table)

In [5]:
Create_Titles_Table='''
Create Table Titles(
	Movie_ID varchar,
	Title_of_the_Movie varchar,
	Movie_or_Show varchar,
	Description varchar,
	Year_of_release varchar,
	Age_certification_PG_rating varchar,
	Runtime_in_minutes varchar,
	Genres varchar,
	Production_countries varchar,
	Number_of_seasons int,
	IMDB_ID varchar,
	IMDB_Score double PRECISION,
	IMDB_Votes int,
	TMDB_Popularity double PRECISION,
	TMDB_Score double PRECISION
)
'''

In [6]:
cur.execute(Create_Titles_Table)

In [7]:
copy_Data_to_Titles='''
copy Titles(
	Movie_ID,
	Title_of_the_Movie,
	Movie_or_Show,
	Description,
	Year_of_release,
	Age_certification_PG_rating,
	Runtime_in_minutes,
	Genres,
	Production_countries,
	Number_of_seasons,
	IMDB_ID,
	IMDB_Score,
	IMDB_Votes,
	TMDB_Popularity,
	TMDB_Score
) from 'C:\\Users\\Vimal\\Desktop\\Projects\\Snowflake Data Warehouse Project\\Data\\Titles_Raw.csv'
Delimiter ','
CSv Header;
'''

In [8]:
cur.execute(copy_Data_to_Titles)

In [9]:
copy_Data_to_Credits='''
copy Credits (
	Person_ID,
	Movie_ID,
	Name_of_the_Person,
	Character_played, 
	Role_of_the_person
) from 'C:\\Users\\Vimal\\Desktop\\Projects\\Snowflake Data Warehouse Project\\Data\\Credits_Raw.csv'
Delimiter ','
CSv Header;'''

In [10]:
cur.execute(copy_Data_to_Credits)

In [16]:
create_IMDB_Table ='''
Create Table IMDB(
	IMDB_ID varchar,
	IMDB_Score double PRECISION,
	IMDB_Votes int
)
'''

In [17]:
create_TMDB_Table='''
Create Table TMDB(
	TMDB_Popularity double PRECISION,
	TMDB_Score double PRECISION
)'''

In [13]:
create_Actors_Table='''
Create Table Actors(
	Person_ID varchar,
	Movie_ID varchar,
	Name_of_the_Person varchar,
	Character_played varchar,
	Role_of_the_person varchar
)'''


In [14]:
create_Directors_Table='''
Create Table Directors(
	Person_ID varchar,
	Movie_ID varchar,
	Name_of_the_Person varchar,
	Role_of_the_person varchar
)
'''

In [18]:
cur.execute(create_IMDB_Table)

In [19]:
cur.execute(create_TMDB_Table)

In [20]:
cur.execute(create_Actors_Table)

DuplicateTable: relation "actors" already exists


In [21]:
cur.execute(create_Directors_Table)

DuplicateTable: relation "directors" already exists


In [22]:
insert_into_IMDB_Table='''
Insert into IMDB
Select IMDB_ID,
IMDB_Score,
IMDB_Votes from Titles'''

In [23]:
cur.execute(insert_into_IMDB_Table)

In [24]:
insert_into_TMDB_Table='''Insert into TMDB
Select TMDB_Popularity,
TMDB_Score from Titles'''

In [25]:
cur.execute(insert_into_TMDB_Table)

In [28]:
insert_into_Actors_Table='''
Insert into Actors
Select 
Person_ID,
Movie_ID,
Name_of_the_Person,
Character_played, 
Role_of_the_person
from Credits where Role_of_the_person='ACTOR'
'''

In [29]:
cur.execute(insert_into_Actors_Table)

In [30]:
insert_into_Directors_Table='''
Insert into Directors
Select 
Person_ID,
Movie_ID,
Name_of_the_Person,
Role_of_the_person 
from Credits where Role_of_the_person='DIRECTOR'
'''

In [31]:
cur.execute(insert_into_Directors_Table)

In [34]:
Copy_credits_and_Titles_to_a_new_csv_file='''
copy (Select  
t.Movie_ID,
t.Movie_or_Show,
t.Year_of_release,
t.Age_certification_PG_rating,
t.Runtime_in_minutes,
t.Genres,
t.Production_countries,
t.Number_of_seasons,
t.IMDB_Score,
t.IMDB_Votes,
t.TMDB_Popularity,
t.TMDB_Score,
c.Name_of_the_Person,
c.Role_of_the_person from Titles t inner join 
Credits c on c.Movie_ID=t.Movie_ID) to 'C:\\Users\\Vimal\\Desktop\\Projects\\Snowflake Data Warehouse Project\\Data\\s3_data\\s3_Warehouse_data.csv'
Delimiter ','
CSV Header'''

In [37]:
cur.execute(Copy_credits_and_Titles_to_a_new_csv_file)